[View in Colaboratory](https://colab.research.google.com/github/mirman-school/project-impact/blob/master/RPS_Data_Template.ipynb)

# RPS Data

Now that we've generated our Rock, Paper, Scissors data, it's time to analyze the results. 

This work is best done in groups of two. **Partner up!**

## Setting up the environment

These next cells handle the business of setting up our notebook to pull data from Google Sheets, and then import the data into a _pandas DataFrame_. Remember those?

For the next six cells, just click **Run** on each cell to get the latest data into our DataFrame. You will be asked at one point to click a link and enter a code. Do so to continue the process.

In [0]:
# Install the library to get Google Sheets into Python
!pip install --upgrade -q gspread

In [0]:
# Authorize this notebook to look at our stuff
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
# Access the spreadsheet holding our results
sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1mPkvMVKIWGtN8XTJ4yhWX6ElX3aRB8LXrXALLQOkzIE/edit#gid=905288117")

In [0]:
# Get rows from sheet
rows = sheet.sheet1.get_all_values()

In [0]:
# Import trusty Pandas and Numpy
import pandas as pd
import numpy as np

In [0]:
df = pd.DataFrame.from_records(rows[1:], columns=rows[0])

## Working with DataFrames

Some of this might be review, but we're going to go through it anyway. There's a lot of ways to interact with DataFrames. Let's start with getting an overview of the data with `df.info()` 

In [0]:
# Call df.info() on the line below


Dang, that's a lot of weird information. Sometimes pandas output can be a little hard to parse. Let's break it down.

### `RangeIndex`

First, it's important to remember that DataFrames are just tables, but represented in code, so they have special abilities. Like a table, it has rows and columns. Generally, columns are just numbered, starting at 0. But sometimes are rows are ordered in a special way, like by date. The way rows are ordered is called an **index**. `RangeIndex` just means we have plain old rows with row numbers starting at 0.

### `Data columns`

You should see 6 columns, each holding some number of `non-null object`s. In our case, the number of entries (rows) should match the number of items in every column, but if there's data missing from a cell, the numbers won't match.

That's enough from `df.info()`. Let's take a quick preview of the data with `df.head()`

In [0]:
# Call df.head() on the next line


So obviously `df.head()` gives us the first 5 lines of data. Wanna see the last 5? Try `df.tail()`

In [0]:
# Call df.tail() to see the last 5 lines of the DataFrame


Sometimes all we _really_ need to know is the number of rows/columns in the DataFrame. For that, we look at `df.shape`. Notice that there are no parentheses after this one. `df.shape` is a **property** of the DataFrame. `head()` and `info()` are **methods**. We'll talk more about the difference soon.

In [0]:
# Retrieve df.shape


`df.shape` is a **tuple**, or a fixed set of values. As you might have guessed, this tuple represents the number of rows and the number of columns in the DataFrame.

### Accessing Columns

What if we just want one of the columns, for example `Player 1 Choice`? To access it, we would put the name of the column in quotes, inside square brackets attached to the DataFrame, like so:

```python
df["Player 1 Choice"]
```

Try it yourself!

In [0]:
# Access the "Player 1 Choice" column


## Exploratory Data Analysis (EDA)

Many of the operations that we will perform on our data can be labeled as **EDA**, or Exploratory Data Analysis. Simply put, we do basic measurements/graphs of the dataset to get a sense of the stories it's telling.

**Discuss with your partner:** What are some basic measurements we could do with our data?

### Counts

Maybe one of you said "We could count how many rocks there were." That's true, we could! To do that, we need to **filter** our DataFrame so that only rows with rocks show are left.

Let's start by filtering the `Player 1 Choice` column for only rocks. To do that, we need to apply a **boolean** statement to the DataFrame.


#### Boolean Statements

Booleans, named after [George Boole](https://en.wikipedia.org/wiki/George_Boole), are statements that are either `True` or `False`.

Try some of these boolean statements in the code cell below

```python
5 > 0
8 == 8
"Hello" != "hello"
True == True
```

In [0]:
# Try one boolean at a time, or multiple by wrapping each one in a print() call


#### Filtering using `[]`

If we can come up with a boolean statement that uses one of our DataFrame's columns, we can put that statement inside of the square braces to make our filter. Keep in mind that, because some of our column names have spaces, we actually have to use square braces to access the columns.

**With your partner, try to come up with a boolean statement that would only catch Player 1 choices that equal rock**.

_Hint: `==` can be used to check if a value matches across an entire column_.

**Once you have it, enter your filter below and get the results**

In [0]:
p1_rock = None # Change None to your filter!
p1_rock # Display the result

#### Using `shape` for counts

Because `shape` returns the rows and columns of a DataFrame, it can tell us the count of values after we've applied a filter. Now, `shape` is a tuple, which works kind of like a list. We only want the first part of the tuple, so we use the zero index to get it. Watch:

In [0]:
shape_example = (10,20) # Notice the parens instead of [] for a tuple
shape_example[0]

With that knowledge, use `shape` to extract the count of Player 1 rock choices from `p1_rock`.

In [0]:
# Use shape and tuple indexing to get just the row count from p1_rock


### Grouping by Player

Our data will become more useful if we group the games by player. Put another way, we want a set of DataFrame, each DataFrame only containing games that involved a specific player. To do that, we're first going to talk about dictionaries

#### Dictionaries

Lists you know. They're just series of values. Dictionaries allow us to store information by _key_, so it's easy to look it up later. Here's an example dictionary.

In [0]:
users = { # Curly brace instead of [
    "steve": "letmein", # key : value
    "alice": "password1",
    "azrael": "123456"
}

# Adding a user to the dict
users["linda"] = "passwordpasswordpassword" # We add to dictionaries by naming the key and assigning with = 

print(users["steve"]) # Change this to see what happens when you use different keys

These are terrible passwords for these users, but as you can see, it's easy to look up a password for a given username. So we're going to build a dictionary where the keys are player names, and the values are DataFrames holding ONLY games that player played in.

#### Multiple Possibilities

How do we know that a player played in a game? If the player is listed as `"Player 1"`, they played. But ALSO if they're listed as `"Player 2"`! So we need to say something like:

_"Get all rows where Player 1 equals the player's name, OR Player 2 equals the player's name."_

Inside our curly braces, we can can combine boolean statements with `&` for AND, and `|` for OR. `&` will require both sides to be true to catch a row, and `|` will only require one side to be true to catch a row. Each side of an `&` or `|` needs to be in parens.

By way of example, here's a filter on our DataFrame that catches when the winning choice was "Rock" or "Scissors":

In [0]:
rock_or_scissors = df[(df["Winner Choice"] == "Rock") | (df["Winner Choice"] == "Scissors")] # Notice the | operator, and how each side is in parentheses
print(rock_or_scissors["Winner Choice"]) # prove we got the right thing by printing the winner choice column

#### Player Names

It all starts with a list of player names. Let's go ahead and build that list. I'll get it started.

In [0]:
# Finish this list so all the players listed in the data are included!
# Make sure you use the names as they appear in the data!

players = [
    "Maceo",
    "Nathaniel",
    "Asha"
]

With a list of players complete, we can loop through them to create both keys and values in our dictionary that will hold DataFrames with only that player's games.

Here we go! Complete this loop to make the `games_by_player` dictionary.

In [0]:
# Create the empty dictionary we intend to fill
games_by_player = {}

# Loop through the player names we build above
for p in players:
  # Create a new key/value pair based on the player's 
  games_by_player[p] = None # Change this to use a two-part filter on our DataFrame with the | operator!
  
 
for g in games_by_player.values():
  print(g.shape) # This won't work until you do the part above

#### Win Counts

With our `players` list, it should easy to get a second dictionary of wins for each player. In fact, this one will be easier than the `games_by_player` dictionary, because we only have to check one column! Let's repeat the process we used above to make a dictionary called `player_wins` with filtered DataFrames with only wins for each players.

In [0]:
# Create the empty dictionary, yet again
player_wins = {}

# Another loop
for p in players:
  player_wins[p] = None # Change this to filter df where the "Winner" column equals the player
  

for w in player_wins:
  print(w, player_wins[w].shape[0]) # This will print wins with the player name


#### Win Percentage

The formula for win percentage is:

$$\frac{w}{g} \times 100$$

where *w* is the number of wins, and *g* is the total number of games. Luckily, we have an easy way to get each from our two existing dictionaries. Time to make a third dictionary, `win_percentages`, that will not hold DataFrames as its values, but the resulting calculation based on values pulled from `games_by_player` and `player_wins`.

**Hint**: `shape` is a kind of list that holds 2 values: rows and columns. `shape[0]` is always going to be the number of rows in a DataFrame.

In [0]:
win_percentages = {}

for p in players:
  total_games = None # Get the number of rows from games_by_player
  wins = None # Get the number of rows from player_wins
  percentage = None # Do the math!
  win_percentages[p] = percentage # Attach the calculated percentage

for wp in win_percentages:
  print(wp, win_percentages[wp]) # Print out the percentages

### Mean (Average)

The mean, or average, value of a set of values is formally defined as:

$$\bar{x} = \frac{1}{n}\left (\sum_{i=1}^n{x_i}\right ) = \frac{x_1+x_2+\cdots +x_n}{n}$$

If you haven't seen that notation before, just breathe. It's okay! The *x* with the bar over it is the symbol for mean. The giant E-looking thing is a sigma, which in math notation indicates a sum. The number on the bottom of the sigma indicates where the count of values starts, and the value on top indicates where it ends. This is all the mathematical "code" for:

_The mean is the sum of all the values in the sample divided by the number of samples_. 

But I wanted you to see the math notation and begin getting used to it. You'll see sigma a lot in this course.

#### Nicer Mean

Now, I bet you could figure out a loop that would allow us to go through each value in `win_percentages` and calculate the mean. Take a moment to think about how you would do that.

Go ahead, I'll wait.

...

...

...

Ready? Cool. It turns out that for operations such as mean, another handy Python library called `numpy` can make life a lot easier. Observe:

In [0]:
# We already imported numpy as np above
scores = [99, 100, 75, 60, 25]

np.mean(scores)

Using what you already know about how to extract values from dictionaries, produce the average win percentage of the class using `np.mean()`.

In [0]:
# Use np.mean() to get the average win percentage out of win_percentages


### Graphing

FINALLY, we're in a position to graph something! Let's try to graph our win percentages. We need 2 axes.

* x-axis: Player names
* y-axis: Win percentages

#### Graphing prerequisites

We need to do a few things before we're ready to graph. The first is import the `matplotlib.pyplot` library, which we'll call `plt`.

In [0]:
import matplotlib.pyplot as plt

Every time we create a graph, we need to call `plt.clf()` to clear any previous graphs from re-rendering. Then we call `plt.figure()` to create a new figure. We can optionally pass a `figsize` argument to `plt.figure()` to make it bigger.

In [0]:
plt.clf()

plt.figure(figsize=[15,10]) # figsize is a list of width, height

Then we build our graph. In this case, it's a bar graph, so we use `plt.bar()`, passing the x and y axes lists.

After that, we use `plt.title()`, `plt.xlabel()`, and `plt.ylabel()` to label the graph.

Finally, we use `plt.show()` to show the graph.

In [0]:
plt.bar([],[]) # Replace the empty lists with the x/y axes we need. HINT: how did we print the games_by_player values?

# Fill in your labels!
plt.title("")
plt.xlabel("")
plt.ylabel("")

# Make it happen!
plt.show()

## Your Turn

It's time to come up with your own EDA questions. List at least one here, then use your Python/Pandas skills to answer your questions!

### My questions:

1. ???